In [1]:
import torch
from torch import nn, optim
from torchvision import transforms, datasets
import random
import math
import numpy as np
from torch.utils.tensorboard import SummaryWriter

### 神经网络有关参数

In [2]:
class Config:
    batch_size = 256
    lr = 0.001
    num_epochs = 10
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    debug = True # 用于控制过程中是否有具体输出

### 加载数据集

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.5],std=[0.5])])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=Config.batch_size, shuffle=False)

### 定义卷积神经网络

In [ ]:
def get_padding_same(map_size, kernel_size, stride):
    """ 在输出特征图大小不变情况下获取padding值 """
    return math.floor(((map_size - 1) * stride + kernel_size - map_size) / 2)

def get_map_size(map_size, kernel_size, stride, padding):
    """ 获取输出的特征图大小 """
    return math.floor((map_size - kernel_size + 2 * padding) / stride + 1)

def CNN(structure: list):
    net = nn.Sequential()
    last_in_channels = 1 # 图片是灰度图片，最初只有一个通道
    last_map_size = 28
    last_in_features = None
    flag = True
    i = 0
    while i < len(structure):
        if structure[i] == 'C':
            # ['C', 卷积核大小, 步长, 输出通道数]
            padding = get_padding_same(last_map_size, structure[i+1], structure[i+2])
            net.append(nn.Conv2d(in_channels= last_in_channels, out_channels=structure[i+3],
                        kernel_size=structure[i+1], stride=structure[i+2], 
                        padding=padding))
            last_in_channels = structure[i+3]
            last_map_size = get_map_size(last_map_size, structure[i+1], structure[i+2], padding)
            net.append(nn.ReLU())
            i += 4

        elif structure[i] == 'P':
            # ['P', 卷积核大小, 步长]
            net.append(nn.MaxPool2d(kernel_size=structure[i+1], stride=structure[i+2]))
            # net.append(nn.BatchNorm2d(num_features=last_in_channels))
            last_map_size = math.floor((last_map_size - structure[i+1]) / structure[i+2] + 1)
            i += 3

        elif structure[i] == 'F':
            # ['F', 输出向量维度]
            if flag:
                # 将数据展开为一维
                net.append(nn.Flatten())
                last_in_features = last_map_size * last_map_size * last_in_channels
                flag = False
        
            net.append(nn.Linear(in_features=last_in_features, out_features=structure[i+1]))
            net.append(nn.ReLU())
            last_in_features = structure[i+1]
            i += 2

    if flag:
        # 将数据展开为一维
        net.append(nn.Flatten())
        last_in_features = last_map_size * last_map_size * last_in_channels
        flag = False
    
    net.append(nn.Linear(in_features=last_in_features, out_features=10))
        
    return net

### 训练和测试函数

In [ ]:
def train_and_test(structure: list):
    net = CNN(structure).to(Config.device)
    # 初始化参数
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, nonlinearity="relu")
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

    optimizer = optim.Adam(net.parameters(), lr=Config.lr)
    loss = nn.CrossEntropyLoss(reduction="sum")
    net.train()
    for e in range(Config.num_epochs):
        for i, (x, y) in enumerate(train_loader):
            x = x.to(Config.device)
            y = y.to(Config.device)
            optimizer.zero_grad()
            output = net(x)
            l = loss(output, y)
            l.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Epoch [{e+1}/{Config.num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {l}')

    # 在测试集上测试效果
    correct_num = 0
    net.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(Config.device)
            y = y.to(Config.device)
            output = net(x)
            correct_num += output.argmax(1).eq(y).sum().item()
    
    del net
    torch.cuda.empty_cache()
        
    return correct_num / len(test_dataset)

### 存储训练过程中的数据

In [ ]:
class Data:
    fitness_sum_list = []
    fitness_max_list = []
    best_individual_list = []
    population_list = [
        [['C', 3, 3, 32, 'P', 2, 1, 'C', 7, 1, 256, 'P', 2, 2, 'F', 256, 'F', 32], ['C', 3, 3, 192, 'P', 4, 1, 'C', 3, 1, 32], ['C', 7, 3, 128, 'F', 256], ['C', 5, 2, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 4, 1, 'P', 4, 2, 'C', 5, 2, 192, 'P', 3, 1], ['C', 5, 3, 64, 'P', 4, 2, 'C', 7, 3, 128, 'P', 2, 1, 'P', 4, 1, 'C', 3, 2, 64], ['C', 3, 3, 64, 'P', 4, 2, 'C', 3, 3, 256], ['C', 3, 1, 256, 'C', 7, 3, 64, 'P', 3, 1, 'P', 4, 1, 'F', 192, 'F', 128], ['C', 3, 2, 192, 'P', 4, 1, 'C', 5, 1, 64, 'F', 192, 'F', 128], ['C', 7, 2, 64, 'F', 32, 'F', 256]],
        [['C', 3, 2, 192, 'C', 5, 1, 64, 'P', 4, 1, 'P', 4, 2, 'C', 5, 2, 192, 'P', 3, 1], ['C', 7, 1, 192, 'P', 4, 1, 'C', 7, 2, 192, 'F', 192, 'F', 128], ['C', 5, 2, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 3, 2, 192, 'P', 4, 1, 'C', 5, 1, 64, 'F', 192, 'F', 128], ['C', 7, 3, 128, 'F', 256], ['C', 3, 3, 32, 'C', 3, 3, 128, 'C', 7, 1, 256, 'P', 2, 2, 'F', 256, 'F', 32], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 4, 1, 'P', 4, 2, 'C', 5, 2, 192, 'P', 3, 1], ['C', 7, 3, 128, 'F', 256], ['C', 5, 2, 192, 'C', 3, 2, 128, 'P', 4, 1, 'P', 4, 2, 'C', 5, 2, 192, 'P', 3, 1], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 2, 2, 'C', 3, 2, 256]],
        [['C', 5, 2, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 3, 128, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 5, 2, 192, 'F', 256], ['C', 7, 1, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 5, 2, 192, 'C', 7, 2, 192, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'P', 4, 1, 'C', 7, 2, 192, 'F', 256, 'F', 128], ['C', 7, 3, 128, 'F', 192], ['C', 7, 1, 192, 'P', 4, 1, 'C', 7, 2, 192, 'F', 192, 'F', 128], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 4, 1, 'P', 4, 2, 'C', 5, 2, 192, 'P', 3, 1]],
        [['C', 7, 1, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'P', 4, 1, 'C', 7, 2, 192, 'F', 256, 'F', 128], ['C', 7, 1, 192, 'P', 4, 1, 'C', 7, 2, 192, 'F', 256, 'F', 128], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 4, 1, 'P', 4, 2, 'C', 5, 2, 192, 'P', 3, 1], ['C', 5, 2, 192, 'C', 7, 2, 192, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 3, 128, 'P', 4, 1, 'C', 3, 2, 128, 'F', 192, 'F', 32], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256]],
        [['C', 7, 1, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'P', 4, 1, 'C', 3, 2, 128, 'F', 256, 'F', 128], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'P', 4, 1, 'C', 7, 2, 192, 'F', 256, 'F', 128], ['C', 7, 1, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 2, 2, 'C', 3, 2, 256]],
    ]
    fitness_list = [
        [0.9816, 0.9555, 0.9834, 0.9871, 0.984, 0.9834, 0.9691, 0.9706, 0.9825, 0.9847],
        [0.9713, 0.9878, 0.9888, 0.9788, 0.9808, 0.9784, 0.9874, 0.9844, 0.9867, 0.9861],
        [0.9867, 0.9888, 0.9853, 0.9802, 0.9879, 0.9884, 0.991, 0.9875, 0.984, 0.9878],
        [0.9915, 0.9892, 0.9901, 0.9889, 0.9851, 0.9907, 0.9906, 0.9902, 0.9913, 0.9913],
        [0.9866, 0.9883, 0.9901, 0.9905, 0.9907, 0.9886, 0.9915, 0.9911, 0.9918, 0.9805],
        [0.9911, 0.9897, 0.9924, 0.9889, 0.9893, 0.9891, 0.9912, 0.9902, 0.9912, 0.989]
    ]
    

### 遗传算法类
* ['C', 卷积核大小, 步长, 输出通道数]
* ['P', 卷积核大小, 步长]
* ['F', 输出向量维度]

In [ ]:
def dprint(input):
    if Config.debug:
        print(input)

In [ ]:
class GN:
    def __init__(self) -> None:
        self.N = 10                                                 # 种群个体数
        self.mutation_rate = 0.1                                    # 变异率
        self.crossover_rate = 0.8                                   # 交叉率
        self.max_generation = 50                                    # 进化的最大代数
        self.target = 0.995                                         # 网络的目标正确识别率

        self.max_cp = 6                                             # 卷积和池化层总数的最大值
        self.max_f = 3                                              # 全连接层个数的最大值
        self.c_kernel_size_choices = [3, 5, 7]                      # 卷积层卷积核大小的选择
        self.c_stride_choices = [1, 2, 3]                           # 卷积层步长的选择
        self.c_channel_num = [32, 64, 128, 192, 256]                # 卷积层输出通道数的选择
        self.p_kernel_size_choices = [2, 3, 4]                      # 池化层卷积核大小的选择
        self.p_stride_choices = [1, 2]                              # 池化层步长的选择
        self.fc_feature_size_choices = [32, 64, 128, 192, 256]      # 全连接层输出大小的选择
        self.type_len = {                                           # 每种网络层的类型和数组元素长度的对应关系
            'C': 4,
            'P': 3,
            'F': 2
        }

    def _gen_C(self):
        """ 生成卷积层 """
        res = []
        kernel_size = random.choice(self.c_kernel_size_choices)
        stride = random.choice(self.c_stride_choices)
        channel_num = random.choice(self.c_channel_num)
        # ['C', 卷积核大小, 步长, 输出通道数]
        res.append('C')
        res.append(kernel_size)
        res.append(stride)
        res.append(channel_num)
        return res

    def _gen_P(self):
        """ 生成池化层 """
        res = []
        kernel_size = random.choice(self.p_kernel_size_choices)
        stride = random.choice(self.p_stride_choices)
        # ['P', 卷积核大小, 步长]
        res.append('P')
        res.append(kernel_size)
        res.append(stride)
        return res
    
    def _gen_F(self):
        """ 生成全连接层 """
        res = []
        feature_size = random.choice(self.fc_feature_size_choices)
        # ['F', 输出向量维度]
        res.append('F')
        res.append(feature_size)
        return res
    
    def _get_structure(self, individual: list):
        """ 
            获取染色体的表示结构，每项的第一个元素表示在数组 indiviaual 中的下标
            {
                'C': [[0, 7, 1, 5], [10, 5, 3, 7], [17, 5, 1, 7]], 
                'P': [[4, 2, 2], [7, 4, 2], [14, 2, 1], [21, 4, 2]], 
                'F': [[24, 320], [26, 128]]
            }
        """
        i = 0
        res = {
            'C': [],
            'P': [],
            'F': []
        }
        while i < len(individual):
            type = individual[i]
            length = self.type_len[type]
            if type == 'C':
                ele = individual[i:i+length]
                ele[0] = i
                res['C'].append(ele)

            elif type == 'P':
                ele = individual[i:i+length]
                ele[0] = i
                res['P'].append(ele)

            elif type == 'F':
                ele = individual[i:i+length]
                ele[0] = i
                res['F'].append(ele)

            i += length
        return res
    
    def _is_vaild(self, individual: list) -> bool:
        """ 判断每个个体的染色体能否正确构成网络 """
        i = 0
        last_map_size = 28
        while i < len(individual):
            type = individual[i]
            if type == 'C':
                padding = get_padding_same(last_map_size, individual[i+1], individual[i+2])
                if 2 * padding + last_map_size < individual[i+1]:
                    return False
                last_map_size = get_map_size(last_map_size, individual[i+1], individual[i+2], padding)
                
            elif type == 'P':
                if last_map_size < individual[i+1]:
                    return False
                last_map_size = get_map_size(last_map_size, individual[i+1], individual[i+2], 0)
                
            elif type == 'F':
                if last_map_size < 1:
                    return False
                break

            i += self.type_len[type]
    
        return True

    def _initialize(self) -> list:
        """ 初始化种群 """
        P = []
        num = 0 # 当前生成的合法个体数
        while num < self.N:
            n_cp = random.randint(1, self.max_cp) # 卷积层和池化层总个数
            p = []
            flag = True # 用于控制第一个层必须为卷积层
            last = None # 用于记录上一个层是卷积层还是池化层
            for _ in range(n_cp):
                r = random.random()
                # 当上一层是卷积层时有0.2的概率继续生成卷积层
                # 当上一层是池化层时有0.8的概率生成卷积层
                if flag or (last == 'C' and r < 0.2) or (last == 'P' and r < 0.8):
                    p.extend(self._gen_C())
                    flag = False
                    last = 'C'
                    
                else:
                    p.extend(self._gen_P())
                    last = 'P'

            
            n_f = random.randint(0, self.max_f - 1) # 全连接层个数
            for _ in range(n_f):
                p.extend(self._gen_F())

            # 判断当前生成的个体是否合法
            if self._is_vaild(p):
                P.append(p)
                num += 1
            else:
                dprint(f'初始化种群时，个体 {p} 不合法，被舍弃，重新生成')

        self.population = P
        return P
    
    def _evaluate(self):
        """ 评估，计算每个个体的适应度 """
        fitness = []
        for i, item in enumerate(self.population):
            dprint(f'第 {i+1} 个个体网络结构 {item}')
            accuracy = train_and_test(item)
            fitness.append(accuracy)
            dprint(f'第 {i+1} 个个体准确率 {accuracy}')
        self.fitness = fitness
        return fitness
    
    def _crossover(self, parent1: list, parent2: list):
        """ 交叉产生下一代 """
        parent1_S = self._get_structure(parent1)
        parent2_S = self._get_structure(parent2)
        child1 = None
        child2 = None

        while True:
            child1 = parent1.copy()
            child2 = parent2.copy()

            for t in ['C', 'P', 'F']:
                r = random.random()
                if r < self.crossover_rate:
                    len1 = len(parent1_S[t])
                    len2 = len(parent2_S[t])
                    len_min = min(len1, len2)
                    # 如果长度为0，则跳过
                    if len_min == 0:
                        continue
                    
                    pos = random.randint(0, len_min) # 交叉的基因个数
                    length = self.type_len[t]
                    for i in range(pos):
                        index1 = parent1_S[t][i][0]
                        index2 = parent2_S[t][i][0]
                        child1[index1:index1+length] = parent2[index2:index2+length]
                        child2[index2:index2+length] = parent1[index1:index1+length]
            
            if self._is_vaild(child1) and self._is_vaild(child2):
                break
            else:
                dprint(f'交叉过程中，子代 {child1} 或子代 {child2} 不合法，被舍弃，重新生成')

        return child1, child2

    def _mutate(self, individual: list):
        """ 对个体进行变异 """
        individual_S = self._get_structure(individual)
        len_total = len(individual_S['C']) + len(individual_S['P']) + len(individual_S['F'])
        new_I = None
        while True:
            new_I = individual.copy()
            r_ = random.random()
            if r_ < self.mutation_rate:
                # 选择某个基因进行变异
                select_gene = random.randint(0, len_total-1)
                select_i = None
                select_type = None
                select_len = None
                i = 0
                temp = 0
                while i < len(individual):
                    type = individual[i]
                    length = self.type_len[type]
                    if temp == select_gene:
                        select_i = i
                        select_type = type
                        select_len = length
                    temp += 1
                    i += length
                
                r = random.random() # 用于选择该基因是改变、删除、增加
                if r < 1 / 3:
                    # 删除基因
                    if select_gene == 0: # 不能删除第一个基因
                        continue
                    
                    new_I = new_I[:select_i] + new_I[select_i+select_len:]
                    dprint(f'delete: before {individual}; after {new_I}')
                    
                elif r < 2 / 3:
                    # 改变基因
                    new = None
                    if select_type == 'C':
                        if select_gene == 0: # 确保第一个层永远为卷积层
                            new = self._gen_C()
                        else:
                            new = self._gen_C() if random.random() < 0.5 else self._gen_P()
                    elif select_type == 'P':
                        new = self._gen_P() if random.random() < 0.5 else self._gen_C()
                    elif select_type == 'F':
                        new = self._gen_F()
                    
                    new_I[select_i:select_i+select_len] = new
                    dprint(f'modify: before {individual}; after {new_I}')

                else:
                    # 增加基因
                    new = None
                    if select_type == 'C':
                        if select_gene == 0: # 确保第一个层永远为卷积层
                            new = self._gen_C()
                        else:
                            new = self._gen_C() if random.random() < 0.5 else self._gen_P()
                    elif select_type == 'P':
                        new = self._gen_P() if random.random() < 0.5 else self._gen_C()
                    elif select_type == 'F':
                        new = self._gen_F()

                    new_I = new_I[:select_i] + new + new_I[select_i:]
                    dprint(f'add: before {individual}; after {new_I}')
                    
            if self._is_vaild(new_I):
                break
            else:
                dprint(f'变异过程中，个体 {new_I} 不合法，被舍弃，重新生成')
                
        return new_I

    def run(self):
        writer = SummaryWriter()

        self._initialize()
        for e in range(self.max_generation):
            self.generation = e
            print(f'\n{'⬇️'*20} 第 {e+1} 代 {'⬇️'*20}\n')
            if e < len(Data.fitness_list):
                self.fitness = Data.fitness_list[e]
            else:
                self._evaluate()
                Data.fitness_list.append(self.fitness)
            
            # fitness = []
            # for i, item in enumerate(self.population):
            #     print(f'第{i+1}个个体 {item}')
            #     fitness.append(random.random())
            #     self.fitness = fitness
            
            best_fitness_i = np.argmax(self.fitness)
            fitness_sum = np.sum(self.fitness)
            best_fitness = self.fitness[best_fitness_i]
            best_individual = self.population[best_fitness_i]
            writer.add_scalar('最大准确率', best_fitness, e+1)
            writer.add_scalar('适应度总和', fitness_sum, e+1)

            Data.fitness_sum_list.append(fitness_sum)
            Data.fitness_max_list.append(best_fitness)
            Data.best_individual_list.append(best_individual)

            print(f'\n{'#'*100}\n# 第 {e+1} 代\n# 种群的总适应度为 {fitness_sum}\n# 最好的网络结构为 {best_individual}\n# 最大正确率为 {best_fitness}\n{'#'*100}\n')
            if best_fitness >= self.target:
                print(f'👍👍👍👍👍 第 {e+1} 代达到目标，迭代停止 👍👍👍👍👍')
                break
            
            next_P = []
            if e < len(Data.population_list):
                next_P = Data.population_list[e]
            else:
                parent_probabilities = self.fitness.copy()
                not_zero_num = 0
                for i in range(len(parent_probabilities)):
                    parent_probabilities[i] -= 0.98
                    if parent_probabilities[i] <= 0:
                        parent_probabilities[i] = 0
                    else:
                        parent_probabilities[i] = math.pow(10, 10 * parent_probabilities[i])                        
                        not_zero_num += 1
                # print(parent_probabilities)
                if not_zero_num < 2:
                    parent_probabilities = self.fitness.copy()
                parent_probabilities = parent_probabilities / np.sum(parent_probabilities)
                # print(parent_probabilities)
                for i in range(0, self.N, 2):
                    # 采用轮盘赌算法选择父体
                    # print(self.population)
                    parent1, parent2 = random.choices(self.population, k=2, weights=parent_probabilities)
                    child1, child2 = self._crossover(parent1, parent2)
                    child1 = self._mutate(child1)
                    child2 = self._mutate(child2)
                    next_P.append(child1)
                    next_P.append(child2)
                Data.population_list.append(next_P)
            
            self.population = next_P
        writer.close()

In [ ]:
gn = GN()
gn.run()


⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️ 第 1 代 ⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️


####################################################################################################
# 第 1 代
# 种群的总适应度为 9.7819
# 最好的网络结构为 ['C', 5, 1, 192, 'P', 4, 2, 'C', 7, 2, 128, 'P', 2, 2, 'P', 3, 1]
# 最大正确率为 0.9871
####################################################################################################


⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️ 第 2 代 ⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️


####################################################################################################
# 第 2 代
# 种群的总适应度为 9.8305
# 最好的网络结构为 ['C', 7, 3, 128, 'F', 256]
# 最大正确率为 0.9888
####################################################################################################


⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️ 第 3 代 ⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️


####################################################################################################
# 第 3 代
# 种群的总适应度为 9.

KeyboardInterrupt: 

In [ ]:
print(Data.population_list)
print(Data.fitness_list)

[[['C', 3, 3, 32, 'P', 2, 1, 'C', 7, 1, 256, 'P', 2, 2, 'F', 256, 'F', 32], ['C', 3, 3, 192, 'P', 4, 1, 'C', 3, 1, 32], ['C', 7, 3, 128, 'F', 256], ['C', 5, 2, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 7, 1, 192, 'C', 7, 2, 192, 'P', 4, 1, 'P', 4, 2, 'C', 5, 2, 192, 'P', 3, 1], ['C', 5, 3, 64, 'P', 4, 2, 'C', 7, 3, 128, 'P', 2, 1, 'P', 4, 1, 'C', 3, 2, 64], ['C', 3, 3, 64, 'P', 4, 2, 'C', 3, 3, 256], ['C', 3, 1, 256, 'C', 7, 3, 64, 'P', 3, 1, 'P', 4, 1, 'F', 192, 'F', 128], ['C', 3, 2, 192, 'P', 4, 1, 'C', 5, 1, 64, 'F', 192, 'F', 128], ['C', 7, 2, 64, 'F', 32, 'F', 256]], [['C', 3, 2, 192, 'C', 5, 1, 64, 'P', 4, 1, 'P', 4, 2, 'C', 5, 2, 192, 'P', 3, 1], ['C', 7, 1, 192, 'P', 4, 1, 'C', 7, 2, 192, 'F', 192, 'F', 128], ['C', 5, 2, 192, 'C', 3, 2, 128, 'P', 2, 2, 'C', 3, 2, 256], ['C', 3, 2, 192, 'P', 4, 1, 'C', 5, 1, 64, 'F', 192, 'F', 128], ['C', 7, 3, 128, 'F', 256], ['C', 3, 3, 32, 'C', 3, 3, 128, 'C', 7, 1, 256, 'P', 2, 2, 'F', 256, 'F', 32], ['C', 7, 1, 192, 'C', 7, 2,

True
